In [7]:
import sys, os, importlib

In [8]:
names = ['numpy', 'scipy', 'matplotlib', 'tqdm', 'PySide2', 'mayavi']
def check_modules(module_names):
    for name in module_names:
        if name in sys.modules:
            print(f"{name!r} already in sys.modules")
        elif (spec := importlib.util.find_spec(name)) is not None:
            # If you choose to perform the actual import ...
            module = importlib.util.module_from_spec(spec)
            sys.modules[name] = module
            spec.loader.exec_module(module)
            print(f"{name!r} has been imported")
        else:
            print(f"can't find the {name!r} module")
check_modules(names)

'numpy' already in sys.modules
'scipy' already in sys.modules
'matplotlib' already in sys.modules
'tqdm' already in sys.modules
'PySide2' already in sys.modules
'mayavi' already in sys.modules


Если какой-то модуль не установлен, то нужно его установить через !pip intsall package. Скорее всего это будет PySide2.
Вроде я указал все, что нужно

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from tqdm import tqdm
from mayavi.mlab import quiver3d
from mayavi import mlab
from scipy.special import lpmv, eval_legendre

In [33]:
g_1_0 = -29404.8 * 1e-9
g_1_1 = -1450.9 * 1e-9
h_1_1 = 4652.5 * 1e-9
g_2_0 = -2499.6 * 1e-9
g_2_1 = 2982.0 * 1e-9
h_2_1 = -2991.6 * 1e-9
g_2_2 = 1677.0 * 1e-9
h_2_2 = -734.6 * 1e-9
Re = 6400 * 1e3
Rm = 11 * Re
b = 5 * Re
mu0=1.26*1e-6
m_x = 4*np.pi*Re**3/mu0*g_1_1*0
m_y = 4*np.pi*Re**3/mu0*h_1_1*0
m_z = 4*np.pi*Re**3/mu0*g_1_0*0
Q_xx = 4*np.pi*Re**4/mu0*(-g_2_0 + np.sqrt(3) * g_2_2*0)
Q_yy = 4*np.pi*Re**4/mu0*(-g_2_0 - np.sqrt(3) * g_2_2*0)
Q_xy = 4*np.pi*np.sqrt(3)*Re**4/mu0*h_2_2*0
Q_xz = 4*np.pi*np.sqrt(3)*Re**4/mu0*g_2_1*0
Q_yz = 4*np.pi*np.sqrt(3)*Re**4/mu0*h_2_1*0

In [34]:
mu0, m_x, m_y, m_z, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz

(1.26e-06,
 -0.0,
 0.0,
 -0.0,
 4.182440109898716e+28,
 4.182440109898716e+28,
 -0.0,
 0.0,
 -0.0)

In [45]:
class Magnetic_field():
    
    def __init__(self, cf=True):
        self.cf = cf
        
    # Поле Земли из диссертации 2004го года
    @staticmethod
    def B_earth_m(x, y, z):
        global b, mu0, m_x, m_y, m_z, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        r = np.sqrt(x**2+y**2+(z+b)**2)
        a_d = -m_x*(z+b)+m_y*y+m_z*x
        a_q = Q_xx*(z+b)**2+Q_yy*y**2-(Q_xx+Q_yy)*x**2-2*Q_xy*(z+b)*y-2*Q_xz*(z+b)*x+2*Q_yz*x*y
        B_x = m_z/r**3-3*x*a_d/r**5-((Q_xx+Q_yy)*x+Q_xz*(z+b)-Q_yz*x)/r**5-5*x*a_q/(2*r**7)
        B_y = m_y/r**3-3*y*a_d/r**5+(Q_yy*y-Q_xy*(z+b)+Q_yz*x)/r**5-5*y*a_q/(2*r**7)
        B_z = -m_x/r**3-3*(z+b)*a_d/r**5+(Q_xx*(z+b)-Q_xy*y-Q_xz*x)/r**5-5*(z+b)*a_q/(2*r**7)
        return -mu0/(4*np.pi)*np.array([B_x, B_y, B_z])

    # Преобразование из сферической системы координат в декртову
    @staticmethod
    def transform(theta, phi):
        C = np.array([[np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)],
                      [np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)],
                      [-np.sin(theta), np.cos(phi), 0]])
        return C

    # Вычисляем суммарное поле(cf+earth)
    def B_xyz_m(self, x, y, z):
        cf = self.cf
        r = np.sqrt(x**2+y**2+z**2)
        theta = np.arccos(z / r)
        phi = np.arctan(y / x) if x > 0 else np.arctan(y / x) + np.pi
        B_earth = self.B_earth_m(x, y, z)
        if cf:
            B_cf_M = self.B_cfi_2004(r, theta, phi)
            C = self.transform(theta, phi)
            B_cf1 = B_cf_M @ C
            B = B_earth + B_cf1
        else:
            B = B_earth
        return B

    # Строим сферу
    @staticmethod
    def plot_sphere(r=6400 * 1e3, x_0=0, y_0=0, z_0=-b):
        [phi, theta] = np.mgrid[0:2 * np.pi:30j, 0:np.pi:30j]
        x = r * np.cos(phi) * np.sin(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(theta)
        mlab.mesh(x + x_0, y + y_0, z + z_0, colormap='gist_earth')

    # Считаем магнитные линии
    def magn_lines_m(self, start_points, step=1e5, R_bound = 6400*1e3*30, max_iter=10_000):
        global Re
        lines = []
        for point in tqdm(start_points, desc='Lines plotted'):
            r, theta, phi = point
            x_gse = r * np.sin(theta) * np.cos(phi)
            y_gse = r * np.sin(theta) * np.sin(phi)
            z_gse = r * np.cos(theta)
            x = z_gse
            y = y_gse
            z = -(x_gse+b)
            line = [np.array([x, y, z])]
            B = self.B_xyz_m(x, y, z)
            x = np.sign(theta - np.pi / 2)
            i = 0
            while Re <= r <= R_bound and i < max_iter:
                new_point = line[-1] + x * step * B / np.linalg.norm(B)
                r = np.linalg.norm(new_point)
                line.append(new_point)
                B = self.B_xyz_m(*new_point)
                i += 1
            lines.append(line)
        return lines

    # Строим магнитные линии
    def plot_magnetic_field_lines_m(self, theta, phi):
        global Re
        cf = self.cf
        if cf:
            color=(0.1, 0.3, 0.5)
        else:
            color=(0.5, 0.3, 0.5)
        start_points = []
        for i in range(theta.shape[0]):
            for j in range(phi.shape[0]):
                theta_ = theta[i]
                phi_ = phi[j]
                point = np.array([Re, theta_, phi_])
                start_points.append(point)
        lines = self.magn_lines_m(start_points)
        for line in lines:
            line = np.array(line)
            if line.shape[0] == 2:
                pass
            else:
                #print(line)
                mlab.plot3d(line[:, 0], line[:, 1], line[:, 2], tube_radius=None, color=color)

        self.plot_sphere()

    # Преобразование из декартовой системы в М-систему, вероятно, не работает...
    @staticmethod
    def decart_to_m(r, theta, phi):
        global b
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        x_m = z
        y_m = y
        z_m = -(x + b)
        r_m = np.sqrt(x_m**2 + y_m**2 + z_m**2)
        theta_m = np.arccos(z_m / r_m)
        phi_m = np.arctan(y_m / x_m) if x_m > 0 else np.arctan(y_m / x_m) + np.pi
        A = np.array([[np.sin(phi_m), -np.cos(phi_m), 0],
                      [np.cos(theta_m)*np.cos(phi_m), np.cos(theta_m)*np.sin(phi_m), -np.sin(theta_m)],
                      [np.sin(theta_m)*np.cos(phi_m), np.sin(theta_m)*np.sin(phi_m), np.cos(theta_m)]])
        return A, r_m, theta_m, phi_m
    
    @staticmethod
    def B_cfi_r(n, r, theta, phi):
        global b, Rm, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        B_r = (n+1)*(-b/Rm)**(n+2)*(r/Rm)**(n-1)*(
                                        -n*(m_x+(n-1)/(4*b)*Q_xx)*lpmv(0, n, np.cos(theta))+
                                        ((m_z+(n-1)/(3*b)*Q_xz)*np.cos(phi)+(m_y+(n-1)/(3*b)*Q_xy)*np.sin(phi))*lpmv(1, n, np.cos(theta))+
                                        1/(6*b)*((0.5*Q_xx+Q_yy)*np.cos(2*phi)-Q_yz*np.sin(2*phi))*lpmv(2, n, np.cos(theta))
                                                  )
        return B_r
    
    @staticmethod
    def B_cfi_theta(n, r, theta, phi):
        global b, Rm, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        B_theta = (n+1)/n*(-b/Rm)**(n+2)*(r/Rm)**(n-1)*(
                            -n*(m_x+(n-1)/(4*b)*Q_xx)*
                            (-((n+1)*1/np.sin(theta)*(np.cos(theta)*eval_legendre(n, np.cos(theta))+eval_legendre(n+1, np.cos(theta)))))+
                            ((m_z+(n-1)/(3*b)*Q_xz)*np.cos(phi)+(m_y+(n-1)/(3*b)*Q_xy)*np.sin(phi))*
                            (-((n+1)*1/np.tan(theta)*lpmv(1, n, np.cos(theta)))+n*1/np.sin(theta)*lpmv(1, n+1, np.cos(theta)))+
                            1/(6*b)*((0.5*Q_xx+Q_yy)*np.cos(2*phi)-Q_yz*np.sin(2*phi))*
                            (-((n+1)*1/np.tan(theta)*lpmv(2, n, np.cos(theta)))+(n-1)*1/np.sin(theta)*lpmv(2, n+1, np.cos(theta)))
                                                        )
        return B_theta
    
    @staticmethod
    def B_cfi_phi(n, r, theta, phi):
        global b, Rm, Q_xx, Q_yy, Q_xy, Q_xz, Q_yz
        B_phi = (n+1)/n*(-b/Rm)**(n+2)*(r/Rm)**(n-1)*1/np.sin(theta)*(
                                        (-(m_z+(n-1)/(3*b)*Q_xz)*np.sin(phi)+(m_y+(n-1)/(3*b)*Q_xy)*np.cos(phi))*lpmv(1, n, np.cos(theta))+
                                        1/(3*b)*(-(0.5*Q_xx+Q_yy)*np.sin(2*phi)-Q_yz*np.cos(2*phi))*lpmv(2, n, np.cos(theta))
                                                  )
        return B_phi
    
    def B_cfi_2004(self, r, theta, phi, N=3):
        global b, mu0
        B_r = 0.0
        B_theta = 0.0
        B_phi = 0.0
        for n in range(1, N+1):
            B_r += self.B_cfi_r(n, r, theta, phi)     
            B_theta += self.B_cfi_phi(n, r, theta, phi)                                              
            B_phi += self.B_cfi_phi(n, r, theta, phi)
        return mu0/(4*np.pi * b**3) * np.array([B_r, B_theta, B_phi])

Посторим поле квадруполя

In [ ]:
theta_1 = [np.pi / x for x in range(4, 11, 1)]
theta_2 = [np.pi/2 + np.pi / x for x in range(9, 34, 3)]
theta = np.concatenate((theta_1, theta_2), axis=None)
phi = np.array([x * np.pi / 6 for x in range(13)])
mf_with_cf = Magnetic_field(cf=True)
mf_without_cf = Magnetic_field(cf=False)
mf_with_cf.plot_magnetic_field_lines_m(theta, phi)
mf_without_cf.plot_magnetic_field_lines_m(theta, phi)
#mlab.plot3d([0, 0], [0, 0], [0, 6400*1e3*10], tube_radius=None, color=(0.7, 0.3, 0.5))
#mlab.plot3d([0, 0], [0, 6400*1e3*10], [0, 0], tube_radius=None, color=(0.7, 0.3, 0.5))
#mlab.plot3d([0, 6400*1e3*10], [0, 0], [0, 0], tube_radius=None, color=(0.7, 0.3, 0.5))
mlab.view(focalpoint=[0, 0, -b])
mlab.show()

Lines plotted: 100%|█████████████████████████████████████████████████████████████████| 208/208 [00:54<00:00,  3.79it/s]
